In [8]:
import numpy as np
import math
import os

def winsorized_minmax(x, lower, upper):
    x_w = max(min(x, upper), lower)
    return (x_w - lower) / (upper - lower)

def invert(x):
    return 1.0 - x

SYMPTOM_MAP = {
    "none": 0.0,
    "mild": 0.3,
    "moderate": 0.6,
    "severe": 1.0
}

# Функция категоризации риска (5 категорий)
def risk_category(score):
    if score < 0.2:
        return "Very Low"
    elif score < 0.4:
        return "Low"
    elif score < 0.6:
        return "Medium"
    elif score < 0.8:
        return "High"
    else:
        return "Very High"


In [10]:
def read_all_patients(filepath):
    patients = []
    current = {}

    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")

    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                if current:
                    patients.append(current)
                    current = {}
                continue
            key, value = line.split("=")
            current[key.strip()] = value.strip()
        if current:
            patients.append(current)
    return patients

DATA_PATH = r"C:\Users\Honor\PROJECT\patients_forecast.txt"
patients = read_all_patients(DATA_PATH)
print(f"Loaded patients: {len(patients)}")


Loaded patients: 10


In [11]:
# Острый риск
def compute_acute_risk(data, iob_max=6.0):
    glucose = float(data["glucose"])
    trend = abs(float(data["trend"]))
    ketones = float(data["ketones"])
    iob = float(data["iob"])
    meal_time = float(data["time_since_meal"])
    symptoms = data["symptoms"].lower()
    hr = float(data["hr"])
    hr_base = float(data["hr_base"])

    r_glucose = winsorized_minmax(glucose, 3.9, 20.0)
    r_trend = winsorized_minmax(trend, 0.0, 0.2)
    r_ketones = winsorized_minmax(ketones, 0.0, 3.0)
    r_iob = winsorized_minmax(iob, 0.0, iob_max)
    meal_scaled = winsorized_minmax(meal_time, 0.0, 4.0)
    r_meal = invert(meal_scaled)
    r_symptoms = SYMPTOM_MAP.get(symptoms, 0.0)
    delta_hr = abs(hr - hr_base)
    r_hr = winsorized_minmax(delta_hr, 0.0, 40.0)

    return np.array([r_glucose, r_trend, r_ketones, r_iob, r_meal, r_symptoms, r_hr])

# Геометрическая площадь
def polygon_area(radii):
    n = len(radii)
    angle = 2 * math.pi / n
    s = 0.0
    for i in range(n):
        s += radii[i] * radii[(i + 1) % n]
    return 0.5 * math.sin(angle) * s

def normalized_risk(area, n):
    max_area = 0.5 * math.sin(2 * math.pi / n) * n
    return area / max_area


In [12]:
X = []
y = []

for p in patients:
    acute_vec = compute_acute_risk(p)
    acute_area = polygon_area(acute_vec)
    acute_score = normalized_risk(acute_area, len(acute_vec))

    features = np.hstack([acute_vec, acute_area])  # 7 признаков + площадь
    X.append(features)
    y.append(acute_score)

X = np.array(X)
y = np.array(y)

print("Feature matrix shape:", X.shape)


Feature matrix shape: (10, 8)


In [13]:
def ridge_fit(X, y, alpha=1.0):
    n_features = X.shape[1]
    I = np.eye(n_features)
    w = np.linalg.inv(X.T @ X + alpha * I) @ X.T @ y
    return w

def ridge_predict(X, w):
    return X @ w

# Обучение
w = ridge_fit(X, y, alpha=1.0)

# Предсказание на обучающем наборе
y_pred = ridge_predict(X, w)


In [14]:
print("\nPatient | True Risk | True Category | Predicted Risk | Predicted Category")
print("----------------------------------------------------------------------")
for i in range(len(y)):
    true_cat = risk_category(y[i])
    pred_cat = risk_category(y_pred[i])
    print(f"{i+1:>7} | {y[i]:.3f}      | {true_cat:>11} | {y_pred[i]:.3f}       | {pred_cat:>13}")

# Средняя абсолютная ошибка
mae = np.mean(np.abs(y - y_pred))
print(f"\nMean Absolute Error (MAE): {mae:.3f}")



Patient | True Risk | True Category | Predicted Risk | Predicted Category
----------------------------------------------------------------------
      1 | 0.082      |    Very Low | 0.116       |      Very Low
      2 | 0.397      |         Low | 0.425       |        Medium
      3 | 0.003      |    Very Low | 0.016       |      Very Low
      4 | 0.829      |   Very High | 0.784       |          High
      5 | 0.076      |    Very Low | 0.120       |      Very Low
      6 | 0.027      |    Very Low | 0.078       |      Very Low
      7 | 0.627      |        High | 0.621       |          High
      8 | 0.009      |    Very Low | 0.029       |      Very Low
      9 | 0.256      |         Low | 0.294       |           Low
     10 | 0.973      |   Very High | 0.901       |     Very High

Mean Absolute Error (MAE): 0.035
